## Fake News Classifier Using Bidirectional LSTM


In [51]:
import pandas as pd

In [52]:
df=pd.read_csv('Data Assets/train.csv')

In [53]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
###Drop Nan Values
df=df.dropna()

In [55]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [56]:
## Get the Dependent features
y=df['label']

In [57]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [58]:
X.shape

(18285, 4)

In [59]:
y.shape

(18285,)

In [61]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [62]:
### Vocabulary size
voc_size=5000

## One hot Representation

In [63]:
messages=X.copy()

In [64]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [65]:
messages.reset_index(inplace=True)

In [66]:
import nltk
import re
from nltk.corpus import stopwords

In [67]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manpr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [77]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
##onehot_repr

## Embedding Representation

In [78]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4296  206 3783]
 [   0    0    0 ... 1839 3910 4895]
 [   0    0    0 ... 4910 3976 2973]
 ...
 [   0    0    0 ... 1186 1738 2777]
 [   0    0    0 ... 1601 3446 4266]
 [   0    0    0 ... 3977 1957 1274]]


In [79]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4586,
       4372, 2499,  983, 4707, 3250, 2315, 4296,  206, 3783])

In [82]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [81]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

C:\Users\manpr\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [83]:
len(embedded_docs),y.shape

(18285, (18285,))

In [84]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [85]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [88]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8614 - loss: 0.2969 - val_accuracy: 0.9163 - val_loss: 0.1958
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9466 - loss: 0.1397 - val_accuracy: 0.9231 - val_loss: 0.1977
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9656 - loss: 0.0989 - val_accuracy: 0.9210 - val_loss: 0.2093
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9750 - loss: 0.0782 - val_accuracy: 0.9180 - val_loss: 0.2566
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9785 - loss: 0.0637 - val_accuracy: 0.9054 - val_loss: 0.2626
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9826 - loss: 0.0528 - val_accuracy: 0.9051 - val_loss: 0.2751
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9873 - loss: 0.0428 - val_accuracy: 0.9138 - val_loss: 0.3485
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9895 - loss: 0.0360 - val_acc

## Performance Metrics And Accuracy

In [89]:
y_pred_prob=model1.predict(X_test)
y_pred1 = (y_pred_prob > 0.5).astype(int)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [90]:
from sklearn.metrics import confusion_matrix

In [91]:
confusion_matrix(y_test,y_pred1)

array([[3081,  338],
       [ 178, 2438]], dtype=int64)

In [92]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9144987572493787

In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3419
           1       0.88      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.91      0.91      6035

